
## Cadastro Ambiental Rural


In [0]:
from pyspark.sql.functions import col
from pyspark.sql.functions import year

import boto3


###Camada Bronze

#####Extraindo dados da landing zone

In [0]:
file_path = "s3://temasambientais/temas_ambientais.csv"
df_temas_ambientais = spark.read.csv(file_path, header=True, inferSchema=True,sep=";")

#####Verificando os nomes das colunas e os tipos de dados de cada coluna

In [0]:
df_temas_ambientais.dtypes


Out[3]: [('uf', 'string'),
 ('municipio', 'string'),
 ('codigo_ibge', 'int'),
 ('area_do_imovel', 'double'),
 ('registro_car', 'string'),
 ('situacao_cadastro', 'string'),
 ('condicao_cadastro', 'string'),
 ('area_liquida', 'double'),
 ('area_remanescente_vegetacao_nativa', 'double'),
 ('area_reserva_legal_proposta', 'double'),
 ('area_preservacao_permanente', 'double'),
 ('area_nao_classificada', 'double'),
 ('solicitacao_adesao_pra', 'string'),
 ('latitude', 'double'),
 ('longitude', 'double'),
 ('data_inscricao', 'timestamp'),
 ('data_alteracao_condicao_cadastro', 'timestamp'),
 ('area_rural_consolidada', 'double'),
 ('area_servidao_administrativa', 'double'),
 ('tipo_imovel_rural', 'string'),
 ('modulos_fiscais', 'double'),
 ('area_uso_restrito', 'double'),
 ('area_reserva_legal_averbada', 'double'),
 ('area_reserva_legal_aprovada_nao_averbada', 'double'),
 ('area_pousio', 'double'),
 ('data_ultima_retificacao', 'timestamp')]

In [0]:
df_temas_ambientais.columns

Out[4]: ['uf',
 'municipio',
 'codigo_ibge',
 'area_do_imovel',
 'registro_car',
 'situacao_cadastro',
 'condicao_cadastro',
 'area_liquida',
 'area_remanescente_vegetacao_nativa',
 'area_reserva_legal_proposta',
 'area_preservacao_permanente',
 'area_nao_classificada',
 'solicitacao_adesao_pra',
 'latitude',
 'longitude',
 'data_inscricao',
 'data_alteracao_condicao_cadastro',
 'area_rural_consolidada',
 'area_servidao_administrativa',
 'tipo_imovel_rural',
 'modulos_fiscais',
 'area_uso_restrito',
 'area_reserva_legal_averbada',
 'area_reserva_legal_aprovada_nao_averbada',
 'area_pousio',
 'data_ultima_retificacao']

#####Verificando quais os dados presentes no arquivo

In [0]:
df_temas_ambientais.limit(5).toPandas().head()

,uf,municipio,codigo_ibge,area_do_imovel,registro_car,situacao_cadastro,condicao_cadastro,area_liquida,area_remanescente_vegetacao_nativa,area_reserva_legal_proposta,...,data_alteracao_condicao_cadastro,area_rural_consolidada,area_servidao_administrativa,tipo_imovel_rural,modulos_fiscais,area_uso_restrito,area_reserva_legal_averbada,area_reserva_legal_aprovada_nao_averbada,area_pousio,data_ultima_retificacao
0,GO,Nazário,5214408,119.6326,GO-5214408-3AEF2043582E40238C0F84A553686CA7,AT,"Analisado com pendências, aguardando retificaç...",119.6326,6.432028,6.4320,...,NaT,112.301149,0.0,IRU,5.4378,0.0,0.000,0.0,0.0,2014-05-07 16:01:44.305
1,SC,Meleiro,4210803,7.5340,SC-4210803-BC127B0EC8DB49AC9D46D723286241A2,AT,"Aguardando análise, não passível de revisão de...",7.5340,5.517854,0.0000,...,NaT,2.014452,0.0,IRU,0.4186,0.0,1.506,0.0,0.0,2014-05-07 16:02:02.915
2,GO,Nova Roma,5214903,19.4883,GO-5214903-7F58049BD79046E9A904CC81C5AC177A,PE,"Analisado com pendências, aguardando retificaç...",19.3600,19.488263,3.8720,...,NaT,0.000000,0.0,IRU,0.2784,0.0,0.000,0.0,0.0,2014-05-07 16:06:15.777
3,GO,Santa Helena de Goiás,5219308,22.9340,GO-5219308-6478196E75CF4F65800ACA0758575820,PE,"Analisado com pendências, aguardando retificação",22.8800,1.502150,1.5028,...,NaT,3.082890,0.0,IRU,1.1467,0.0,0.000,0.0,0.0,2014-05-07 17:49:36.938
4,PR,Cornélio Procópio,4106407,10.9560,PR-4106407-0F06081500254BE3A479EE8EFFDD5319,AT,Em análise,10.9560,0.000000,0.0000,...,NaT,10.794267,0.0,IRU,0.6087,0.0,0.000,0.0,0.0,2014-05-07 17:52:55.333


#####Persistindo os dados na camada bronze

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS bronze

In [0]:
def create_table(df, format_type, schema_name, table_name, location):
    schema = ", ".join([f"{field.name} {field.dataType.simpleString()}" for field in df.schema.fields])

    spark.sql(f'''CREATE TABLE IF NOT EXISTS {schema_name}.{table_name} ({schema})
              USING {format_type}
              LOCATION '{location}'
              ''')

    df.write.format(format_type) \
       .mode("overwrite") \
       .save(location)

In [0]:
create_table(df_temas_ambientais,'parquet','bronze','temas_ambientais_bronze', '/FileStore/bronze')

In [0]:
%fs ls /FileStore/bronze

path,name,size,modificationTime
dbfs:/FileStore/bronze/_SUCCESS,_SUCCESS,0,1716253079000
dbfs:/FileStore/bronze/_committed_5879178511176830258,_committed_5879178511176830258,2028,1716253079000
dbfs:/FileStore/bronze/_started_5879178511176830258,_started_5879178511176830258,0,1716252974000
dbfs:/FileStore/bronze/part-00000-tid-5879178511176830258-67789624-7e2f-409b-86cb-258c76da8efd-304-1-c000.snappy.parquet,part-00000-tid-5879178511176830258-67789624-7e2f-409b-86cb-258c76da8efd-304-1-c000.snappy.parquet,44897147,1716253071000
dbfs:/FileStore/bronze/part-00001-tid-5879178511176830258-67789624-7e2f-409b-86cb-258c76da8efd-298-1-c000.snappy.parquet,part-00001-tid-5879178511176830258-67789624-7e2f-409b-86cb-258c76da8efd-298-1-c000.snappy.parquet,44961539,1716253068000
dbfs:/FileStore/bronze/part-00002-tid-5879178511176830258-67789624-7e2f-409b-86cb-258c76da8efd-290-1-c000.snappy.parquet,part-00002-tid-5879178511176830258-67789624-7e2f-409b-86cb-258c76da8efd-290-1-c000.snappy.parquet,44818226,1716253018000
dbfs:/FileStore/bronze/part-00003-tid-5879178511176830258-67789624-7e2f-409b-86cb-258c76da8efd-297-1-c000.snappy.parquet,part-00003-tid-5879178511176830258-67789624-7e2f-409b-86cb-258c76da8efd-297-1-c000.snappy.parquet,45067859,1716253025000
dbfs:/FileStore/bronze/part-00004-tid-5879178511176830258-67789624-7e2f-409b-86cb-258c76da8efd-302-1-c000.snappy.parquet,part-00004-tid-5879178511176830258-67789624-7e2f-409b-86cb-258c76da8efd-302-1-c000.snappy.parquet,44844893,1716253072000
dbfs:/FileStore/bronze/part-00005-tid-5879178511176830258-67789624-7e2f-409b-86cb-258c76da8efd-305-1-c000.snappy.parquet,part-00005-tid-5879178511176830258-67789624-7e2f-409b-86cb-258c76da8efd-305-1-c000.snappy.parquet,44932070,1716253074000
dbfs:/FileStore/bronze/part-00006-tid-5879178511176830258-67789624-7e2f-409b-86cb-258c76da8efd-306-1-c000.snappy.parquet,part-00006-tid-5879178511176830258-67789624-7e2f-409b-86cb-258c76da8efd-306-1-c000.snappy.parquet,44945846,1716253079000


### Camada Silver

#####Carregando dados da camada bronze

In [0]:
def load_table(schema_name, table_name):
    full_table_name = f"{schema_name}.{table_name}"
    df = spark.read.table(full_table_name)
    return df



In [0]:
df_temas_ambientais = load_table('bronze', 'temas_ambientais_bronze')

df_temas_ambientais.head(5)

Out[46]: [Row(uf='GO', municipio='Caiapônia', codigo_ibge=5204409, area_do_imovel=197.5993, registro_car='GO-5204409-79418BE625514399A28DC302DC17F013', situacao_cadastro='PE', condicao_cadastro='Analisado com pendências, aguardando retificação e/ou apresentação de documentos', area_liquida=197.2709, area_remanescente_vegetacao_nativa=197.599297358608, area_reserva_legal_proposta=39.5022, area_preservacao_permanente=83.6170568057138, area_nao_classificada=0.0, solicitacao_adesao_pra='Nao', latitude=-17.1671264858525, longitude=-51.4378006049413, data_inscricao=datetime.datetime(2014, 5, 11, 11, 16, 43, 969000), area_rural_consolidada=0.0, area_servidao_administrativa=0.0, tipo_imovel_rural='IRU', area_uso_restrito=0.0, area_pousio=0.0, data_ultima_retificacao=datetime.datetime(2014, 5, 11, 11, 16, 43, 969000), ano_inscricao=2014),
 Row(uf='GO', municipio='Santa Isabel', codigo_ibge=5219357, area_do_imovel=11.0505, registro_car='GO-5219357-7E2B0D59151D4A61ACF787E6CDB3F49B', situacao_cada

In [0]:
df_temas_ambientais.limit(10).toPandas().head()

,uf,municipio,codigo_ibge,area_do_imovel,registro_car,situacao_cadastro,condicao_cadastro,area_liquida,area_remanescente_vegetacao_nativa,area_reserva_legal_proposta,...,latitude,longitude,data_inscricao,area_rural_consolidada,area_servidao_administrativa,tipo_imovel_rural,area_uso_restrito,area_pousio,data_ultima_retificacao,ano_inscricao
0,GO,Caiapônia,5204409,197.5993,GO-5204409-79418BE625514399A28DC302DC17F013,PE,"Analisado com pendências, aguardando retificaç...",197.2709,197.599297,39.5022,...,-17.167126,-51.437801,2014-05-11 11:16:43.969,0.000000,0.0000,IRU,0.0,0.0,2014-05-11 11:16:43.969,2014
1,GO,Santa Isabel,5219357,11.0505,GO-5219357-7E2B0D59151D4A61ACF787E6CDB3F49B,PE,"Analisado com pendências, aguardando retificaç...",11.0498,3.989424,2.2087,...,-15.373476,-49.387997,2014-05-19 08:57:32.380,6.748917,0.0000,IRU,0.0,0.0,2014-05-19 08:57:32.380,2014
2,MG,Santana do Manhuaçu,3158904,65.7500,MG-3158904-E4A1B494C2A34F9597676DD16774E75D,AT,"Aguardando análise, não passível de revisão de...",65.7500,25.330000,25.3300,...,-20.054759,-41.969105,2014-05-22 09:01:17.577,40.400000,0.0000,IRU,0.0,0.0,2014-05-22 09:01:17.577,2014
3,MG,Passos,3147907,66.5000,MG-3147907-9773480ED1EA4894A24A0B8301678AB5,AT,"Aguardando análise, não passível de revisão de...",66.5000,18.150000,18.1500,...,-20.911177,-46.568829,2014-05-29 09:04:26.237,44.810000,0.0000,IRU,0.0,0.0,2014-05-29 09:04:26.237,2014
4,GO,Vila Propício,5222302,3.6300,GO-5222302-004E3C4E538041DAB06D53BD5C798C98,AT,"Analisado com pendências, aguardando retificação",3.4234,1.119167,0.6845,...,-15.350129,-49.034737,2014-05-29 13:24:33.472,2.245002,0.2077,IRU,0.0,0.0,2014-05-29 13:24:33.472,2014


In [0]:
columns_to_drop = ['data_alteracao_condicao_cadastro', 'modulos_fiscais','area_reserva_legal_averbada', 'area_reserva_legal_aprovada_nao_averbada']
df_temas_ambientais = df_temas_ambientais.drop(*columns_to_drop)

#####Tratamento Dados

In [0]:
df_temas_ambientais.count()

Out[9]: 6839104

In [0]:
df_temas_ambientais = df_temas_ambientais.drop_duplicates()
df_temas_ambientais.count()

Out[10]: 6839100

In [0]:
df_temas_ambientais.filter(col("data_inscricao").isNull()).toPandas()

,uf,municipio,codigo_ibge,area_do_imovel,registro_car,situacao_cadastro,condicao_cadastro,area_liquida,area_remanescente_vegetacao_nativa,area_reserva_legal_proposta,...,solicitacao_adesao_pra,latitude,longitude,data_inscricao,area_rural_consolidada,area_servidao_administrativa,tipo_imovel_rural,area_uso_restrito,area_pousio,data_ultima_retificacao
0,GO,Santa Bárbara de Goiás,5219100,6.9875,GO-5219100-9C2FDEFBB58B4695AA92FCD659EB0094,AT,"Aguardando análise, não passível de revisão de...",6.9875,0.058005,0.0580,...,Nao,-16.574055,-49.673627,NaT,6.387876,0.000000,IRU,0.0,0.0,2015-12-15 10:40:01.997
1,PR,Maripá,4115358,5.2525,PR-4115358-BB7CCCF5EB0D48C9A7C6173942766F9C,AT,"Aguardando análise, não passível de revisão de...",5.2525,1.326165,0.0000,...,Nao,-24.391595,-53.765329,NaT,3.674385,0.000000,IRU,0.0,0.0,2015-12-15 13:51:01.709
2,PE,Santa Filomena,2612554,16.4470,PE-2612554-E538440C0E884365A48AE0F70321FC6D,AT,"Aguardando análise, não passível de revisão de...",16.4470,5.469457,3.4252,...,Sim,-8.228723,-40.661958,NaT,10.977518,0.000000,IRU,0.0,0.0,2015-12-15 11:00:48.270
3,PR,São Pedro do Iguaçu,4125753,4.4803,PR-4125753-49312E58E65B43D5B651A6A2D74F417E,AT,"Aguardando análise, não passível de revisão de...",4.4803,1.037868,0.0000,...,Nao,-24.958925,-53.700504,NaT,3.001480,0.000000,IRU,0.0,0.0,2015-12-15 14:04:08.693
4,PR,Guarapuava,4109401,20.3838,PR-4109401-0EA6A1EE0B0E474B95748969A0C42FA0,AT,"Aguardando análise, não passível de revisão de...",20.3838,2.301779,2.3018,...,Sim,-25.464702,-51.745105,NaT,18.071428,0.000000,IRU,0.0,0.0,2015-12-15 13:19:08.511
5,PR,Ubiratã,4128005,22.5041,PR-4128005-6AA3862805FE4379B73051E718E0CA9B,AT,"Aguardando análise, não passível de revisão de...",22.5041,0.730043,0.7300,...,Sim,-24.424462,-53.032777,NaT,21.633508,0.000000,IRU,0.0,0.0,2015-12-15 11:54:32.284
6,PR,Ampére,4101002,7.1003,PR-4101002-253E4A517ECF4C02B9963AFFBA72786A,AT,"Aguardando análise, não passível de revisão de...",7.0368,0.760083,0.4921,...,Nao,-25.957922,-53.529474,NaT,6.005326,0.063518,IRU,0.0,0.0,2015-12-15 13:50:55.327
7,GO,Goianésia,5208608,401.4349,GO-5208608-213BBE0F2E944FF8962628517A6849F4,AT,"Aguardando análise, não passível de revisão de...",401.4349,373.320726,80.2871,...,Nao,-15.561377,-49.193244,NaT,23.811648,0.000000,IRU,0.0,0.0,2015-12-15 14:05:50.736
8,SC,Concórdia,4204301,20.9761,SC-4204301-4D246918B6FB495EA22CF7A993C7EDAB,AT,"Aguardando análise, não passível de revisão de...",20.3807,0.943300,0.9076,...,Nao,-27.238264,-52.110766,NaT,18.843823,0.595324,IRU,0.0,0.0,2016-04-14 16:51:50.885
9,GO,Bom Jesus de Goiás,5203500,280.0801,GO-5203500-7DF1ED8AD9E54427B14D66D2B0B3D711,AT,"Aguardando análise, não passível de revisão de...",280.0801,38.851597,38.8463,...,Sim,-18.199628,-49.608582,NaT,239.063627,0.000000,IRU,0.0,0.0,2016-04-26 08:14:09.734


In [0]:
df_temas_ambientais.count()

Out[12]: 6839100

In [0]:
df_temas_ambientais = df_temas_ambientais.where(df_temas_ambientais.data_inscricao.isNotNull())

In [0]:
df_temas_ambientais.count()

#####Escrever os DFs em tabelas Delta Lake com um projeto de particionamento efetivo conforme o contexto empresarial

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS silver

In [0]:
def create_table_with_partition(df, format_type, partition_by, schema_name, table_name, location):
    partition_clause = ", ".join(partition_by)

    schema = ", ".join([f"{field.name} {field.dataType.simpleString()}" for field in df.schema.fields])

    spark.sql(f'''CREATE TABLE IF NOT EXISTS {schema_name}.{table_name} ({schema})
              USING {format_type}
              PARTITIONED BY ({partition_clause})
              LOCATION '{location}'
              ''')

    df.write.format(format_type) \
       .mode("overwrite") \
       .partitionBy(partition_by) \
       .save(location)

In [0]:
%fs ls /FileStore/silver

Criada a coluna "ano_inscricao" a partir da data_inscricao para ser utilizado na partição

In [0]:
df_temas_ambientais = df_temas_ambientais.withColumn("ano_inscricao", year("data_inscricao"))


In [0]:

create_table_with_partition(df_temas_ambientais, 'DELTA', ['uf','ano_inscricao'], 'silver', 'temas_ambientais', '/FileStore/silver/temas_ambientais/')

 Tabela Delta Lake criadas na camada silver: 
 1. temas_ambientais, particionado por 'uf' e 'ano_inscricao'

### Camada Gold

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS gold

##### Tabela de "Temas Ambientais" por Região do BR

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.temas_ambientais_por_regiao AS
SELECT *,
  CASE 
    WHEN uf IN ('SP', 'RJ', 'MG', 'ES') THEN 'Sudeste'
    WHEN uf IN ('PR', 'RS', 'SC') THEN 'Sul'
    WHEN uf IN ('GO', 'MT', 'MS', 'DF') THEN 'Centro-Oeste'
    WHEN uf IN ('AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO') THEN 'Norte'
    WHEN uf IN ('AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE') THEN 'Nordeste'
    ELSE 'Outra'
  END AS regiao
FROM silver.temas_ambientais;

#### Tabela de Propriedades com Área Remanescente de Vegetação Nativa por UF



In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.propriedades_area_nativa_uf AS
SELECT uf, COUNT(*) AS num_propriedades_area_nativa
FROM silver.temas_ambientais
WHERE area_remanescente_vegetacao_nativa > 0
GROUP BY uf;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-641900871737069>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-641900871737069>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("Q1JFQVRFIFRBQkxFIElGIE5PVCBFWElTVFMgZ29sZC5wcm9wcmllZGFkZXNfYXJlYV9uYXRpdmFfdWYgQVMKU0VMRUNUIHVmLCBDT1VOVCgqKSBBUyBudW1fcHJvcHJpZWRhZGVzX2FyZWFfbmF0aXZhCkZST00gc2lsdmVyLnRlbWFzX2FtYmllbnRhaXMKV0hFUkUgYXJlYV9yZW1hbmVzY2VudGVfdmVnZXRhY2FvX25hdGl2YSA+IDAKR1JPVVAgQlkgdWY=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()

In [0]:
%fs ls dbfs:/user/hive/warehouse/gold.db

### Consultas Analíticas (Contexto Empresarial)

1. Recupere a soma de área (em hectares) para todas as propriedades agrícolas que
pertencem ao MS e MT. Ordene os resultados em ordem decrescente.


In [0]:
%sql
SELECT uf, SUM(area_do_imovel) AS soma_area
FROM silver.temas_ambientais
WHERE uf IN ('MS', 'MT')
GROUP BY uf
ORDER BY soma_area DESC;


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2597619522200211>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-2597619522200211>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("U0VMRUNUIHVmLCBTVU0oYXJlYV9kb19pbW92ZWwpIEFTIHNvbWFfYXJlYQpGUk9NIHNpbHZlci50ZW1hc19hbWJpZW50YWlzCldIRVJFIHVmIElOICgnTVMnLCAnTVQnKQpHUk9VUCBCWSB1ZgpPUkRFUiBCWSBzb21hX2FyZWEgREVTQw==").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(


2. Filtre todas as propriedades que pertencem a região sudeste.


In [0]:
%sql
SELECT *
FROM gold.temas_ambientais_por_regiao
WHERE regiao = 'Sudeste'
LIMIT 10;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-641900871737074>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-641900871737074>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("U0VMRUNUICoKRlJPTSBnb2xkLnRlbWFzX2FtYmllbnRhaXNfcG9yX3JlZ2lhbwpXSEVSRSByZWdpYW8gPSAnU3VkZXN0ZScKTElNSVQgMTA=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter(

3. Calcule quantas propriedades foram cadastradas por ano. Apresente os resultados em
ordem cronológica.


In [0]:
%sql
SELECT ano_inscricao, COUNT(*) AS num_cadastros
FROM silver.temas_ambientais
GROUP BY ano_inscricao
ORDER BY ano_inscricao;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-641900871737076>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-641900871737076>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("U0VMRUNUIGFub19pbnNjcmljYW8sIENPVU5UKCopIEFTIG51bV9jYWRhc3Ryb3MKRlJPTSBzaWx2ZXIudGVtYXNfYW1iaWVudGFpcwpHUk9VUCBCWSBhbm9faW5zY3JpY2FvCk9SREVSIEJZIGFub19pbnNjcmljYW8=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         mo

4. Calcule o percentual médio de área remanescente de vegetação nativa em comparação à área total da propriedade.


In [0]:
%sql
SELECT AVG(area_remanescente_vegetacao_nativa  / area_do_imovel) * 100 AS percentual_medio
FROM silver.temas_ambientais;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-641900871737078>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-641900871737078>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("U0VMRUNUIEFWRyhhcmVhX3JlbWFuZXNjZW50ZV92ZWdldGFjYW9fbmF0aXZhICAvIGFyZWFfZG9faW1vdmVsKSAqIDEwMCBBUyBwZXJjZW50dWFsX21lZGlvCkZST00gc2lsdmVyLnRlbWFzX2FtYmllbnRhaXM=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module

5. Construa uma consulta que mostre a contagem de propriedades rurais por estado.


In [0]:
%sql
SELECT uf, count (*) AS num_propriedades
FROM silver.temas_ambientais
GROUP BY uf
ORDER BY num_propriedades DESC;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-641900871737080>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-641900871737080>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("U0VMRUNUIHVmLCBjb3VudCAoKikgQVMgbnVtX3Byb3ByaWVkYWRlcwpGUk9NIHNpbHZlci50ZW1hc19hbWJpZW50YWlzCkdST1VQIEJZIHVmCk9SREVSIEJZIG51bV9wcm9wcmllZGFkZXMgREVTQw==").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, c

6. Faça a média de área entre todas as propriedades. Calcule quantas propriedades por
estado que estão acima da média.

In [0]:
%sql
WITH media_area AS (
  SELECT AVG(area_do_imovel) AS media_area
  FROM silver.temas_ambientais
)
SELECT 
  t.uf, 
  COUNT(t.registro_car) AS num_propriedades_acima_media
FROM 
  silver.temas_ambientais t
CROSS JOIN 
  media_area
WHERE 
  t.area_do_imovel > media_area.media_area
GROUP BY 
  t.uf
ORDER BY num_propriedades_acima_media;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-641900871737082>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-641900871737082>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("V0lUSCBtZWRpYV9hcmVhIEFTICgKICBTRUxFQ1QgQVZHKGFyZWFfZG9faW1vdmVsKSBBUyBtZWRpYV9hcmVhCiAgRlJPTSBzaWx2ZXIudGVtYXNfYW1iaWVudGFpcwopClNFTEVDVCAKICB0LnVmLCAKICBDT1VOVCh0LnJlZ2lzdHJvX2NhcikgQVMgbnVtX3Byb3ByaWVkYWRlc19hY2ltYV9tZWRpYQpGUk9NIAogIHNpbHZlci50ZW1hc19hbWJpZW50YWlzIHQKQ1JPU1MgSk9JTiAKICBtZWRpYV9hcmVhCldIRVJFIAogIHQuYXJlYV9kb19pbW92ZWwgPiBtZWRpYV9hcmVhLm1lZGlhX2FyZWEKR1JPVVAgQlkgCiAgdC51ZgpPUkRFUiBCWSBudW1fcHJvcHJpZWRhZGVzX2FjaW1hX21lZGlh").decode())
      5   display(df)

7.  Comparar o número de propriedades com área remanescente de vegetação nativa com o número total de propriedades por UF.

In [0]:
%sql
SELECT pn.uf,
       pn.num_propriedades_area_nativa AS num_area_nativa,
       COUNT(t.registro_car) AS num_total_propriedades,
       (num_area_nativa / num_total_propriedades) * 100 AS porcentagem_area_nativa
FROM gold.propriedades_area_nativa_uf pn
JOIN silver.temas_ambientais t
ON pn.uf = t.uf
GROUP BY pn.uf, pn.num_propriedades_area_nativa
ORDER BY pn.num_propriedades_area_nativa DESC;


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-641900871737086>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-641900871737086>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("U0VMRUNUIHBuLnVmLAogICAgICAgcG4ubnVtX3Byb3ByaWVkYWRlc19hcmVhX25hdGl2YSBBUyBudW1fYXJlYV9uYXRpdmEsCiAgICAgICBDT1VOVCh0LnJlZ2lzdHJvX2NhcikgQVMgbnVtX3RvdGFsX3Byb3ByaWVkYWRlcywKICAgICAgIChudW1fYXJlYV9uYXRpdmEgLyBudW1fdG90YWxfcHJvcHJpZWRhZGVzKSAqIDEwMCBBUyBwb3JjZW50YWdlbV9hcmVhX25hdGl2YQpGUk9NIGdvbGQucHJvcHJpZWRhZGVzX2FyZWFfbmF0aXZhX3VmIHBuCkpPSU4gc2lsdmVyLnRlbWFzX2FtYmllbnRhaXMgdApPTiBwbi51ZiA9IHQudWYKR1JPVVAgQlkgcG4udWYsIHBuLm51bV9wcm9wcmllZGFkZXNfYXJlYV9uYXRpdmEKT1JERVIgQlkgcG